In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv", nrows=10000)
df_test = pd.read_csv("test.csv", nrows=10000)

In [3]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [4]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# adding features
from __future__ import print_function
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors

# store smiles
smiles = df_all.smiles

carbons = []
carbons = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('c')))
df_all['carbons'] = pd.DataFrame(carbons)

double = []
double = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('=')))
df_all['double'] = pd.DataFrame(double)

single = []
single = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('-')))
df_all['single'] = pd.DataFrame(single)

nitrogen = []
nitrogen = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('n')))
df_all['nitrogen'] = pd.DataFrame(nitrogen)

oxygen = []
oxygen = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('o')))
df_all['oxygen'] = pd.DataFrame(oxygen)

ccccc = []
ccccc = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('ccccc')))
df_all['ccccc'] = pd.DataFrame(ccccc)

ccc = []
ccc = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('ccc')))
df_all['ccc'] = pd.DataFrame(ccc)

Si = []
Si = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('Si')))
df_all['Si'] = pd.DataFrame(Si)

se = []
se = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('se')))
df_all['se'] = pd.DataFrame(se)

mols = df_all.smiles.astype(str).apply(lambda x: Chem.MolFromSmiles(x))
df_all['mols'] = mols

atoms = np.vstack(df_all.mols.apply(lambda x: x.GetNumAtoms()))
df_all['atoms'] = pd.DataFrame(atoms)
bonds = np.vstack(df_all.mols.apply(lambda x: x.GetNumBonds()))
df_all['bonds'] = pd.DataFrame(bonds)

# aromaticity
aro = []
aro = np.vstack(df_all.mols.apply(lambda x: sum(int(x.GetAtomWithIdx(i).GetIsAromatic()) for i in range(x.GetNumAtoms()))))
df_all['aro'] = pd.DataFrame(aro)

# adding new features

# TPSA BAAAAAD
# tpsa = np.vstack(df_all.smiles.apply(lambda x: Descriptors.TPSA(Chem.MolFromSmiles(x))))
# df_all['tpsa'] = pd.DataFrame(tpsa)

# sp3 hybridization
sp3 = np.vstack(smiles.apply(lambda x: rdMolDescriptors.CalcFractionCSP3(Chem.MolFromSmiles(x))))
df_all['sp3'] = pd.DataFrame(sp3)


#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
df_all = df_all.drop(['mols'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print("Train features:", X_train.shape)
print("Train gap:", Y_train.shape)
print("Test features:", X_test.shape)

Train features: (10000, 269)
Train gap: (10000,)
Test features: (10000, 269)


In [7]:
hd = np.vstack(mols.apply(lambda x: Descriptors.NumHDonors(x)))
df_all['hd'] = pd.DataFrame(hd)

ha = np.vstack(mols.apply(lambda x: Descriptors.NumHAcceptors(x)))
df_all['ha'] = pd.DataFrame(ha)

aliphcarbo = np.vstack(mols.apply(lambda x: Descriptors.NumAliphaticCarbocycles(x)))
df_all['aliphcarbo'] = pd.DataFrame(aliphcarbo)

aliphhetero = np.vstack(mols.apply(lambda x: Descriptors.NumAliphaticHeterocycles(x)))
df_all['aliphhetero'] = pd.DataFrame(aliphhetero)

aliphrings = np.vstack(mols.apply(lambda x: Descriptors.NumAliphaticRings(x)))
df_all['aliphrings'] = pd.DataFrame(aliphrings)

arocarbos = np.vstack(mols.apply(lambda x: Descriptors.NumAromaticCarbocycles(x)))
df_all['arocarbos'] = pd.DataFrame(arocarbos)

aroheteros = np.vstack(mols.apply(lambda x: Descriptors.NumAromaticHeterocycles(x)))
df_all['aroheteros'] = pd.DataFrame(aroheteros)

arorings = np.vstack(mols.apply(lambda x: Descriptors.NumAromaticRings(x)))
df_all['arorings'] = pd.DataFrame(arorings)

heteros = np.vstack(mols.apply(lambda x: Descriptors.NumHeteroatoms(x)))
df_all['heteros'] = pd.DataFrame(heteros)

radelecs = np.vstack(mols.apply(lambda x: Descriptors.NumRadicalElectrons(x)))
df_all['radelecs'] = pd.DataFrame(radelecs)

rotbonds = np.vstack(mols.apply(lambda x: Descriptors.NumRotatableBonds(x)))
df_all['rotbonds'] = pd.DataFrame(rotbonds)

satcarbos = np.vstack(mols.apply(lambda x: Descriptors.NumSaturatedCarbocycles(x)))
df_all['satcarbos'] = pd.DataFrame(satcarbos)

satheteros = np.vstack(mols.apply(lambda x: Descriptors.NumSaturatedHeterocycles(x)))
df_all['satheteros'] = pd.DataFrame(satheteros)

satrings = np.vstack(mols.apply(lambda x: Descriptors.NumSaturatedRings(x)))
df_all['satrings'] = pd.DataFrame(satrings)

valelecs = np.vstack(mols.apply(lambda x: Descriptors.NumValenceElectrons(x)))
df_all['valelecs'] = pd.DataFrame(valelecs)

In [8]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor


# fn that returns trained linear function to make predictions
def train_and_test_linear_model(X_train, Y_train):
    LR = LinearRegression()
    LR.fit(X_train, Y_train)
    # return the prediction we've trained
    return LR.predict



# RIDGE REG
def train_and_test_ridge_model(alpha):
    def helper(X_train, Y_train):
        reg = linear_model.Ridge(alpha=alpha)
        reg.fit(X_train, Y_train)
        return reg.predict
    return helper


# LASSO REG
def train_and_test_lasso_model(alpha):
    def helper(X_train, Y_train):
        reg = linear_model.Lasso(alpha=alpha)
        reg.fit(X_train, Y_train)
        return reg.predict
    return helper


# ELASTICNET REG
def train_and_test_elastic_net_model(alpha):
    def helper(X_train, Y_train):
        reg = linear_model.ElasticNet(alpha=alpha)
        reg.fit(X_train, Y_train)
        return reg.predict
    return helper

# SVM REG
def train_and_test_svm_model():
    def helper(X_train, Y_train):
        reg = SVR(kernel='rbf', C=1.0, epsilon=0.2)
        reg.fit(X_train, Y_train)
        return reg.predict
    return helper

# RF pred
# fn that returns trained Random Forest to make predictions
def train_and_test_RF_model(X_train, Y_train):
    RF = RandomForestRegressor()
    RF.fit(X_train, Y_train)
    # return the prediction we've trained
    return RF.predict

In [9]:
from training_and_validation import select_best_model

In [10]:
select_best_model(X_train, Y_train, 4, train_and_test_linear_model)

next validation set
training


/Users/williamdeuschle/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
Final error 0.22250720290115356


0.22250720290115356

In [11]:
select_best_model(X_train, Y_train, 4, train_and_test_ridge_model(0.1))

next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
Final error 0.22246216384369197


0.22246216384369197

In [12]:
select_best_model(X_train, Y_train, 4, train_and_test_lasso_model(0.1))

next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
Final error 0.3218036125706949


0.3218036125706949

In [13]:
select_best_model(X_train, Y_train, 4, train_and_test_elastic_net_model(0.1))

next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
Final error 0.29505507412140336


0.29505507412140336

In [14]:
select_best_model(X_train, Y_train, 4, train_and_test_svm_model())

next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
Final error 0.20335733899827216


0.20335733899827216

In [15]:
select_best_model(X_train, Y_train, 4, train_and_test_RF_model)

next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
next validation set
training
got a prediction!
Final error 0.2079858841404748


0.2079858841404748

In [ ]:
mols = pd.DataFrame(mols)

In [ ]:
df_all['ha']

In [ ]:
mols.smiles

In [16]:
df_all.head()

,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,feat_010,...,arocarbos,aroheteros,arorings,heteros,radelecs,rotbonds,satcarbos,satheteros,satrings,valelecs
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0,6,6,8,0,3,0,0,0,130
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1,2,3,4,0,1,0,0,0,118
2,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,2,3,5,6,0,1,0,0,0,128
3,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,2,2,4,4,0,1,0,0,0,128
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,2,5,7,9,0,1,0,0,0,136
